In [1]:
# Works on TPUs
import tensorflow as tf

# tpu_name = ''
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_name)

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)
# Initialize GPU devices
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
  print("No GPU devices found. Using CPU.")
  strategy = tf.distribute.OneDeviceStrategy(device='/cpu:0')
else:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')  # Adjust the device name as needed

# Create a distribution strategy for GPUs
strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')  # Adjust the device name as needed

tf.random.set_seed(103847532)

2023-09-08 09:58:31.988754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 09:58:32.213875: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 09:58:33.160808: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-0

# Model Definitions

In [2]:
"""A collection of Neural Net models to be used for experiments."""

from typing import Callable, Optional

from absl import app


def _get_mobilenet(depth_multiplier, num_classes, width_multiplier=1.0):

  """Loads mobilenet.

  Args:
    depth_multiplier: the depth_multiplier parameter for mobilenet
    num_classes: the number of classes
    width_multiplier: the width_multiplier for mobilenet

  Returns:
    Returns the mobilenet model.
  """
  model = tf.keras.applications.mobilenet.MobileNet(
      input_shape=(32, 32, 3),
      alpha=width_multiplier,
      depth_multiplier=depth_multiplier,
      dropout=0.001,
      include_top=True,
      weights=None,
      input_tensor=None,
      pooling=None,
      classes=num_classes,
      classifier_activation='softmax')
  return model


# Implementation of ResNet models that can be found in
# https://keras.io/zh/examples/cifar10_resnet/
def _resnet_layer(inputs,
                  num_filters=16,
                  kernel_size=3,
                  strides=1,
                  activation='relu',
                  batch_normalization=True,
                  conv_first=True):
  """2D Convolution-Batch Normalization-Activation stack builder.

  Args:
    inputs: input tensor from input image or previous layer
    num_filters: Conv2D number of filters
    kernel_size: Conv2D square kernel dimensions
    strides: Conv2D square stride dimensions
    activation: activation name
    batch_normalization: whether to include batch normalization
    conv_first: conv-bn-activation (True) or bn-activation-conv (False)

  Returns:
   x: tensor as input to the next layer
  """

  conv = tf.keras.layers.Conv2D(
      num_filters,
      kernel_size=kernel_size,
      strides=strides,
      padding='same',
      kernel_initializer='he_normal',
      kernel_regularizer=tf.keras.regularizers.l2(1e-4))
  x = inputs
  if conv_first:
    x = conv(x)
    if batch_normalization:
      x = tf.keras.layers.BatchNormalization()(x)
    if activation is not None:
      x = tf.keras.layers.Activation(activation)(x)
  else:
    if batch_normalization:
      x = tf.keras.layers.BatchNormalization()(x)
    if activation is not None:
      x = tf.keras.layers.Activation(activation)(x)
    x = conv(x)
  return x


def _resnet_v2(
    input_shape=(32, 32, 3), depth=29, num_classes=10, data_augmentation=False):

  """ResNet Version 2 Model builder.

  Args:
    input_shape: shape of input image tensor
    depth: number of core convolutional layers
    num_classes: number of classes
    data_augmentation: A boolean variable that determines whether we use data
      augmentation or not

  Returns:
    model (Model): Keras model instance
  """

  if (depth - 2) % 9 != 0:
    raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')

  # Start model definition.
  num_filters_in = 16
  num_res_blocks = int((depth - 2) / 9)
  inputs = tf.keras.layers.Input(shape=input_shape)

  # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
  if data_augmentation:
    data_augmentation_module = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(
            3. / 32, 3. / 32),
    ])
    x = data_augmentation_module(inputs)
    x = _resnet_layer(
        inputs=x, num_filters=num_filters_in, conv_first=True)
  else:
    x = _resnet_layer(
        inputs=inputs, num_filters=num_filters_in, conv_first=True)

  # Instantiate the stack of residual units
  for stage in range(3):
    for res_block in range(num_res_blocks):
      activation = 'relu'
      batch_normalization = True
      strides = 1
      if stage == 0:
        num_filters_out = num_filters_in * 4
        if res_block == 0:  # first layer and first stage
          activation = None
          batch_normalization = False
      else:
        num_filters_out = num_filters_in * 2
        if res_block == 0:  # first layer but not first stage
          strides = 2  # downsample

      # bottleneck residual unit
      y = _resnet_layer(
          inputs=x,
          num_filters=num_filters_in,
          kernel_size=1,
          strides=strides,
          activation=activation,
          batch_normalization=batch_normalization,
          conv_first=False)
      y = _resnet_layer(inputs=y, num_filters=num_filters_in, conv_first=False)
      y = _resnet_layer(
          inputs=y,
          num_filters=num_filters_out,
          kernel_size=1,
          conv_first=False)
      if res_block == 0:
        # linear projection residual shortcut connection to match
        # changed dims
        x = _resnet_layer(
            inputs=x,
            num_filters=num_filters_out,
            kernel_size=1,
            strides=strides,
            activation=None,
            batch_normalization=False)
      x = tf.keras.layers.add([x, y])
      print(num_filters_in, num_filters_out)
    num_filters_in = num_filters_out

  # Add classifier on top.
  # v2 has BN-ReLU before Pooling
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.AveragePooling2D(pool_size=8)(x)
  y = tf.keras.layers.Flatten()(x)
  outputs = tf.keras.layers.Dense(
      num_classes, activation='softmax', kernel_initializer='he_normal')(
          y)

  # Instantiate model.
  model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
  return model


def load_model(model_architecture: str,
               num_classes: int,
               optimizer_name: str,
               loss_function: Optional[Callable[
                   ..., tf.Tensor]] = tf.keras.losses.CategoricalCrossentropy(),
               learning_rate: Optional[float] = 0.001,
               width_multiplier: Optional[float] = 1.0,
               depth_multiplier: Optional[int] = 1,
               resnet_depth: Optional[int] = 11) -> tf.keras.Model:

  """Loads a compiled model.

  Args:
    model_architecture: The name of the model architecture.
    num_classes: The number of classes.
    optimizer_name: The name of the optimizer we use.
    loss_function: The loss function used in distillation.
    learning_rate: The initial learning rate for our optimizer.
    width_multiplier: The width_multiplier of the base_CNN/mobilenet model.
    depth_multiplier: The depth_multiplier of the mobilenet model.
    resnet_depth: The depth of the resnet network which must be of the form 9*d
      + 2 for some integer d, e.g., 11, 20.
  Returns:
    A compiled model to be trained.
  """

  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  if model_architecture == 'mobilenet':
    model = _get_mobilenet(
        depth_multiplier=depth_multiplier,
        num_classes=num_classes,
        width_multiplier=width_multiplier)
  elif model_architecture == 'resnet':
    model = _resnet_v2(depth=resnet_depth, num_classes=num_classes)
  else:
    raise app.UsageError('Invalid argument to --model')

  if optimizer_name == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  elif optimizer_name == 'adagrad':
    optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  elif optimizer_name == 'SGD':
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate, momentum=0.2, nesterov=False)
  else:
    print('Default settings: Adam')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  model.compile(
      loss=loss_function, optimizer=optimizer, metrics='categorical_accuracy')

  return model


In [3]:
model = load_model(model_architecture='resnet', resnet_depth = 20, optimizer_name = 'adam', num_classes = 100)
model.summary()

16 64
16 64
64 128
64 128
128 256
128 256
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)  

# Data Utils

In [13]:
"""Contains some simple data structures for passing around dataset splits."""

import dataclasses
import datetime as dt
from typing import Optional, Tuple

import numpy as np


@dataclasses.dataclass
class LabeledExamples:
  """A LabeledExample dataclass to make passing dataset splits around easier.

    Attributes
      ----------
      examples: tf.Tensor
          A tensor containing the examples (x) of the data.
      labels: tf.Tensor
          A tensor containing the labels (y) of the data.
      size: int
          The number of examples.
      num_classes: int
          The number of different labels.
      trainable: bool
          If true this dataset split is ok to train on (e.g. should be set to
          False for the test split).
  """

  examples: tf.Tensor
  labels: tf.Tensor
  size: int
  num_classes: int
  trainable: bool

  def shuffle(self, seed: Optional[int] = None):
    """Shuffles the order of the examples."""

    if seed is None:
      rand_seed = int(dt.datetime.now().strftime('%f')[:-5])
    else:
      rand_seed = seed

    shuffled_examples = np.random.RandomState(seed=rand_seed).permutation(
        self.examples)
    shuffled_labels = np.random.RandomState(seed=rand_seed).permutation(
        self.labels)
    self.examples = shuffled_examples
    self.labels = shuffled_labels


def split(data: LabeledExamples,
          index: int) -> Tuple[LabeledExamples, LabeledExamples]:
  """Splits a Labeled Examples instance into two parts."""

  split_a_examples = data.examples[:-index]
  spit_a_labels = data.labels[:-index]

  split_a = LabeledExamples(
      split_a_examples,
      spit_a_labels,
      size=len(split_a_examples),
      num_classes=data.num_classes,
      trainable=True)

  split_b_examples = data.examples[-index:]
  split_b_labels = data.labels[-index:]

  split_b = LabeledExamples(
      split_b_examples,
      split_b_labels,
      size=len(split_b_examples),
      num_classes=data.num_classes,
      trainable=True)
  return split_a, split_b


@dataclasses.dataclass
class DataSplit:
  """Class for the dataset splits used in unlabeled distillation.

  Attributes
    ----------
    dataset_a: LabeledExamples
        An instance of LabeledExamples containing the (small) set of labeled
        training data used to train the teacher.
    dataset_b: LabeledExamples
        An instance of LabeledExamples containing the (large) set of training
        data used to train the teacher.  The labels of these points should *not*
        be used to train the student model.
    test: LabeledExamples
        An instance of LabeledExamples containing the test set for evaluating
        the teacher and student models.
    validtion: LabeledExamples, Optional
        An instance of LabeledExamples containing the validation.

  """
  dataset_a: LabeledExamples
  dataset_b: LabeledExamples
  test: LabeledExamples
  validation: Optional[LabeledExamples] = None


In [14]:
"""Methods for loading datasets."""

from typing import Optional

import tensorflow_datasets as tfds


def _example_label_split(dataset):
  x, y = zip(*dataset)
  return tf.convert_to_tensor(x, dtype=tf.float32), tf.convert_to_tensor(y)


def _normalize_x(x):
  x /= 127.5
  x -= 1.0
  return x


def _normalize_y(y, num_classes):
  return tf.keras.utils.to_categorical(y, num_classes)


def load_cifar_data(labeled_percentage: Optional[int] = 20,
                    num_classes: Optional[int] = 10,
                    binary: Optional[bool] = False) -> DataSplit:
  """Loads cifar10 or cifar100 dataset."""

  if num_classes == 10:
    train1, train2, test = tfds.load(
        'cifar10',
        split=[
            f'train[:{labeled_percentage}%]', f'train[{labeled_percentage}%:]',
            'test'
        ],
        as_supervised=True)
  else:
    train1, train2, test = tfds.load(
        'cifar100',
        split=[
            f'train[:{labeled_percentage}%]', f'train[{labeled_percentage}%:]',
            'test'
        ],
        as_supervised=True)

  x_train1, y_train1 = _example_label_split(train1)
  x_train2, y_train2 = _example_label_split(train2)
  x_test, y_test = _example_label_split(test)

  if binary:
    y_train1 = y_train1 % 2
    y_train2 = y_train2 % 2
    y_test = y_test % 2
    num_classes = 2

  x_train1 = _normalize_x(x_train1)
  y_train1 = _normalize_y(y_train1, num_classes)

  x_train2 = _normalize_x(x_train2)
  y_train2 = _normalize_y(y_train2, num_classes)

  x_test = _normalize_x(x_test)
  y_test = _normalize_y(y_test, num_classes)

  dataset_a = LabeledExamples(
      examples=x_train1,
      labels=y_train1,
      size=len(x_train1),
      num_classes=num_classes,
      trainable=True)

  dataset_b = LabeledExamples(
      examples=x_train2,
      labels=y_train2,
      size=len(x_train2),
      num_classes=num_classes,
      trainable=True)

  test = LabeledExamples(
      examples=x_test,
      labels=y_test,
      size=len(x_test),
      num_classes=num_classes,
      trainable=False)

  data_split = DataSplit(
      dataset_a=dataset_a, dataset_b=dataset_b, validation=None, test=test)

  return data_split


def load_celeb_a_data(
    labeled_percentage: int,
    num_classes: Optional[int] = 2,
    label_key: Optional[str] = 'Male',
    group_key: Optional[str] = 'Young') -> DataSplit:
  """Loads celeb_a."""

  def _get_image_and_label(feat_dict):
    return (feat_dict['image'], feat_dict['attributes'][label_key])

  def _preprocess_input_dict(feat_dict):
    # Separate out the image and target variable from the feature dictionary.
    image = feat_dict['image']
    label = feat_dict['attributes'][label_key]
    group = feat_dict['attributes'][group_key]

    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [32, 32])

    label = tf.cast(label, tf.float32)
    group = tf.cast(group, tf.float32)

    feat_dict['image'] = image
    feat_dict['attributes'][label_key] = label
    feat_dict['attributes'][group_key] = group

    return feat_dict

  celeb_a_builder = tfds.builder('celeb_a')
  celeb_a_builder.download_and_prepare()

  train1, train2, test = celeb_a_builder.as_dataset(split=[
      f'train[:{labeled_percentage}%]', f'train[{labeled_percentage}%:]', 'test'
  ])

  x_train1, y_train1 = _example_label_split(
      train1.batch(1).map(_preprocess_input_dict).map(_get_image_and_label))
  x_train2, y_train2 = _example_label_split(
      train2.batch(1).map(_preprocess_input_dict).map(_get_image_and_label))
  x_test, y_test = _example_label_split(
      test.batch(1).map(_preprocess_input_dict).map(_get_image_and_label))

  x_train1 = tf.squeeze(x_train1)
  y_train1 = tf.squeeze(y_train1)

  x_train2 = tf.squeeze(x_train2)
  y_train2 = tf.squeeze(y_train2)

  x_test = tf.squeeze(x_test)
  y_test = tf.squeeze(y_test)

  x_train1 = _normalize_x(x_train1)
  y_train1 = _normalize_y(y_train1, num_classes)

  x_train2 = _normalize_x(x_train2)
  y_train2 = _normalize_y(y_train2, num_classes)

  x_test = _normalize_x(x_test)
  y_test = _normalize_y(y_test, num_classes)

  dataset_a = LabeledExamples(
      examples=x_train1,
      labels=y_train1,
      size=len(x_train1),
      num_classes=num_classes,
      trainable=True)

  dataset_b = LabeledExamples(
      examples=x_train2,
      labels=y_train2,
      size=len(x_train2),
      num_classes=num_classes,
      trainable=True)

  test = LabeledExamples(
      examples=x_test,
      labels=y_test,
      size=len(x_test),
      num_classes=num_classes,
      trainable=False)

  data_split = DataSplit(
      dataset_a=dataset_a, dataset_b=dataset_b, validation=None, test=test)

  return data_split


def load_data(dataset_name: str,
              labeled_percentage: int) -> DataSplit:
  """Loads either cifar10, cifar100, or celeb_a."""

  if dataset_name == 'cifar10':
    num_classes = 10
    return load_cifar_data(
        labeled_percentage=labeled_percentage, num_classes=num_classes)
  elif dataset_name == 'cifar10bin':
    num_classes = 10
    return load_cifar_data(
        labeled_percentage=labeled_percentage,
        num_classes=num_classes,
        binary=True)
  elif dataset_name == 'cifar100':
    num_classes = 100
    return load_cifar_data(
        labeled_percentage=labeled_percentage, num_classes=num_classes)
  elif dataset_name == 'celeb_a':
    num_classes = 2
    return load_celeb_a_data(
        labeled_percentage=labeled_percentage, num_classes=num_classes)
  else:
    raise NameError('Wrong dataset name.')


# Training Methods

In [15]:
"""A function that trains models with or without data augmentation."""

from typing import Optional

import tensorflow as tf


def _train_model(
    model: tf.keras.Model,
    train_x: tf.Tensor,
    train_y: tf.Tensor,
    test_x: tf.Tensor,
    test_y: tf.Tensor,
    data_augmentation: Optional[bool] = False,
    weights: Optional[tf.Tensor] = None,
    epochs: Optional[int] = 200,
    with_lr_scheduler: Optional[bool] = True,
    batch_size: Optional[int] = 128,
    epochs_offset: Optional[int] = 0) -> tf.keras.callbacks.History:
  """Trains a model.

  Args:
    model: Instance of tf.keras.Model.
    train_x: A dataset containing the data to train on.
    train_y: A dataset containing the labels of the data to train on.
    test_x: A dataset containing the data to test on.
    test_y: A dataset containing the labels of the data to test on.
    data_augmentation: True if data augmentation is used and False otherwise.
    weights: The weight sample-weights to be used.
    epochs: The number of epochs to train for.
    with_lr_scheduler: Whether we use a schedule for the learning-rate or not.
    batch_size: The batch size used for training.
    epochs_offset: How many epochs we assume we have performed so far.

  Returns:
      history: History of trained model.

  Raises:
    NotImplementedError: if online data augmention is used with weights or if
    an augmentation method other than None, 'offline', or 'online' is provided.
    To use weights with online data augmentation, weights should be passed as
    an extra advice label and a corresponding loss with advice should be used,
    see the loss functions defined in loss_functions.py.
  """

  def lr_schedule(epoch):
    lr = 1e-3
    if epoch + epochs_offset > 180:
      lr *= 0.5e-3
    elif epoch + epochs_offset > 160:
      lr *= 1e-3
    elif epoch + epochs_offset > 120:
      lr *= 1e-2
    elif epoch + epochs_offset > 80:
      lr *= 1e-1
    return lr

  if with_lr_scheduler:
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
    callbacks = [lr_scheduler]

  with_weights = False
  if weights is not None:
    with_weights = True
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_x, train_y, weights))
  else:
    with_weights = False
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_x, train_y))

  def prepare(ds, shuffle=False, augment=False, with_weights=False):
    autotune = tf.data.AUTOTUNE

    data_augmentation_layer = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomTranslation(
            height_factor=0.1,
            width_factor=0.1,
            fill_mode='nearest',
            interpolation='bilinear',
            seed=None,
            fill_value=0.0)
    ])

    if shuffle:
      ds = ds.shuffle(len(train_x), reshuffle_each_iteration=True)

    # Use data augmentation only on the training set.
    if augment:
      if with_weights:
        ds = ds.map(
            lambda x, y, w: (data_augmentation_layer(x, training=True), y, w),
            num_parallel_calls=autotune)
      else:
        ds = ds.map(
            lambda x, y: (data_augmentation_layer(x, training=True), y),
            num_parallel_calls=autotune)

    # Batch the dataset. The drop_remainder is needed to avoid uneven batches.
    ds = ds.batch(batch_size, drop_remainder=True)

    # Use buffered prefetching on all datasets.
    ds = ds.prefetch(buffer_size=autotune)

    return ds

  train_dataset = prepare(
      train_dataset,
      shuffle=True,
      augment=data_augmentation,
      with_weights=with_weights)

  history = model.fit(
      train_dataset,
      validation_data=(test_x, test_y),
      epochs=epochs,
      verbose=1,
      workers=4,
      callbacks=callbacks)

  return history


def train_model(
    model: tf.keras.Model,
    train_x: tf.Tensor,
    train_y: tf.Tensor,
    test_x: tf.Tensor,
    test_y: tf.Tensor,
    data_augmentation: Optional[str] = None,
    weights: Optional[tf.Tensor] = None,
    epochs: Optional[int] = 200,
    with_lr_scheduler: Optional[bool] = True,
    batch_size: Optional[int] = 128,
    epochs_offset: Optional[int] = 0) -> tf.keras.callbacks.History:
  """Trains a model.

  Args:
    model: Instance of tf.keras.Model.
    train_x: A dataset containing the data to train on.
    train_y: A dataset containing the labels of the data to train on.
    test_x: A dataset containing the data to test on.
    test_y: A dataset containing the labels of the data to test on.
    data_augmentation: Can be either 'no', 'offline', 'online'.
    weights: The weight sample-weights to be used.
    epochs: The number of epochs to train for.
    with_lr_scheduler: Whether we use a schedule for the learning-rate or not.
    batch_size: The batch size used for training.
    epochs_offset: How many epochs we assume we have performed so far.

  Returns:
      history: History of trained model.
  """

  if data_augmentation != 'online':

    if data_augmentation == 'no' or not data_augmentation:
      use_data_augmentation = False
    else:
      use_data_augmentation = True

    return _train_model(
        model=model,
        train_x=train_x,
        train_y=train_y,
        test_x=test_x,
        test_y=test_y,
        epochs=epochs,
        data_augmentation=use_data_augmentation,
        weights=weights,
        with_lr_scheduler=with_lr_scheduler,
        batch_size=batch_size,
        epochs_offset=epochs_offset)

  elif data_augmentation == 'online':
    results = []

    for i in range(1, epochs+1):

      print(f'Epoch {i}/{epochs}')

      history_tmp = _train_model(
          model=model,
          train_x=train_x,
          train_y=train_y,
          test_x=test_x,
          test_y=test_y,
          epochs=1,
          data_augmentation=True,
          weights=weights,
          with_lr_scheduler=with_lr_scheduler,
          batch_size=batch_size,
          epochs_offset=i)

      results.append(history_tmp)

    history = results[0]

    for hist in results[1:]:
      for key in hist.history.keys():
        history.history[key] += hist.history[key]
    return history

  else:
    raise NotImplementedError()


# Experiment Parameters

In [16]:
# Dataset used for training.
_DATASET =  'cifar100'
# Whether the teacher outputs soft-labels or hard-labels.
_WITH_SOFT_LABELS = True
# Whether we pretrain the student model.
_WITH_PRETRAINING = True
# The model architecture we are using for the teacher.
_TEACHER_MODEL = 'resnet'
# The data augmentation method we are using for the teacher.
_TEACHER_DATA_AUGMENTATION =  'online'
# The data augmentation method we are using for pretraining the student.
_STUDENT_PRETRAINING_DATA_AUGMENTATION = 'online'
# Data augmentation method we are using for training the student during
_STUDENT_DISTILLATION_DATA_AUGMENTATION =  'online'
# Student model architecture.
_STUDENT_MODEL =  'resnet'
# Teacher mobilenet depth multiplier.
_TEACHER_MOBILENET_DEPTH_MULTIPLIER = 2
# Student mobilenet depth multiplier.
_STUDENT_MOBILENET_DEPTH_MULTIPLIER = 1
# Teacher resnet depth.
_TEACHER_RESNET_DEPTH = 110
# Student resnet depth.
_STUDENT_RESNET_DEPTH = 56
# Student training epochs.
_STUDENT_EPOCHS = 100
# Teacher training epochs.
_TEACHER_EPOCHS = 200
# Student pretraining epochs.
_PRETRAINING_EPOCHS = 200
# Student optimizer.
_STUDENT_OPTIMIZER = 'adam'
# Teacher optimizer.
_TEACHER_OPTIMIZER = 'adam'
# Size of validation dataset.
_VALIDATION_DATASET_SIZE = 512
# Size of labeled dataset (percentage of labeled examples).
_SIZE_OF_DATASET_A = 25
# Whether to soft-label the labeled dataset A.
_SOFT_LABEL_DATASET_A = False
# Number of experiment trials.
_NUM_TRIALS =  3
# Whether to randomize the dataset in each trial.
_RANDOMIZE_DATASET = True
# Whether to train on validation data.
_TRAIN_ON_VALIDATION = True
# The batch size to be used.
_BATCH_SIZE = 128
# The random seed to be used.
_SEED = 753410
# The path to load/save models.
_CNS_PATH = '/cns/jv-d/home/kontonis/colab/'


# The filenames of the pretrained student model.
student_load_temp_filename = 'pretrained_weights.h5'
student_save_temp_filename = 'pretrained_weights.h5'

# Load the Dataset

In [17]:
# Load the dataset
data_split = load_data(
    dataset_name=_DATASET, labeled_percentage=_SIZE_OF_DATASET_A)

# Number of classes should be equal for all three splits (dataset_a,
# dataset_b, test) so we set it using the labeled split, dataset_a.
num_classes = data_split.dataset_a.num_classes


# Train/Load the Teacher Model

In [18]:
import os
# from google3.pyglib import gfile

# Training the teacher
teacher_params = {
    'dataset': _DATASET,
    'epochs': _TEACHER_EPOCHS,
    'optimizer': _TEACHER_OPTIMIZER,
    'labeled_percentage': _SIZE_OF_DATASET_A,
    'architecture': _TEACHER_MODEL,
    'data_augmentation': _TEACHER_DATA_AUGMENTATION
    }

if _TEACHER_MODEL == 'mobilenet':
  teacher_params['depth'] = _TEACHER_MOBILENET_DEPTH_MULTIPLIER
elif _TEACHER_MODEL == 'resnet':
  teacher_params['depth'] = _TEACHER_RESNET_DEPTH

teacher_filename = 'TEACHER' + str(teacher_params)

teacher_filepath = os.path.join(_CNS_PATH, teacher_filename)


teacher_load_temp_filename = 'temp_teacher_load.h5'
teacher_save_temp_filename = 'temp_teacher_save.h5'

# Training the teacher
with strategy.scope():
  teacher_model = load_model(
        _TEACHER_MODEL,
        num_classes,
        _TEACHER_OPTIMIZER,
        width_multiplier=1.0,
        depth_multiplier=_TEACHER_MOBILENET_DEPTH_MULTIPLIER,
        resnet_depth=_TEACHER_RESNET_DEPTH)

# if gfile.Exists(teacher_filepath):
#   gfile.Copy(teacher_filepath, teacher_load_temp_filename, overwrite=True)
#   with strategy.scope():
#     teacher_model.load_weights(teacher_load_temp_filename)
# else:
_ = train_model(
      teacher_model,
      data_split.dataset_a.examples,
      data_split.dataset_a.labels,
      data_split.test.examples,
      data_split.test.labels,
      data_augmentation=_TEACHER_DATA_AUGMENTATION,
      epochs=_TEACHER_EPOCHS)

  # # If the provided CNS path is not empty try to save the teacher model.
  # if _CNS_PATH:
  #   with strategy.scope():
  #     teacher_model.save_weights(teacher_save_temp_filename)
  #   gfile.Copy(teacher_save_temp_filename, teacher_filepath, overwrite=True)


Epoch 1/200


2023-09-04 07:47:14.617809: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:1771"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

97/97 [==============================] - 464s 1s/step - loss: 6.0996 - categorical_accuracy: 0.0780 - val_loss: 6.2722 - val_categorical_accuracy: 0.0472 - lr: 0.0010
Epoch 2/200


2023-09-04 07:55:34.799150: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:1809"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

97/97 [==============================] - 74s 760ms/step - loss: 5.2968 - categorical_accuracy: 0.1299 - val_loss: 5.3481 - val_categorical_accuracy: 0.0977 - lr: 0.0010
Epoch 3/200


...

97/97 [==============================] - 22s 221ms/step - loss: 0.4708 - categorical_accuracy: 1.0000 - val_loss: 3.0814 - val_categorical_accuracy: 0.5468 - lr: 5.0000e-07
Epoch 199/200


2023-09-04 09:32:43.496075: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:9295"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

97/97 [==============================] - 22s 231ms/step - loss: 0.4708 - categorical_accuracy: 0.9999 - val_loss: 3.0821 - val_categorical_accuracy: 0.5462 - lr: 5.0000e-07
Epoch 200/200


2023-09-04 09:33:06.539515: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:9333"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

97/97 [==============================] - 21s 221ms/step - loss: 0.4705 - categorical_accuracy: 0.9999 - val_loss: 3.0818 - val_categorical_accuracy: 0.5464 - lr: 5.0000e-07


In [19]:
score = teacher_model.evaluate(data_split.test.examples,
                                data_split.test.labels)

if isinstance(score, (list, tuple, np.ndarray)):
  teacher_accuracy = score[1] * 100.0
print("The teacher's accuracy is:", teacher_accuracy)

313/313 [==============================] - 9s 28ms/step - loss: 3.0818 - categorical_accuracy: 0.5464
The teacher's accuracy is: 54.64000105857849


In [20]:
# # saving the teacher model with 0.15 labeled data
# tf.saved_model.save(teacher_model, f"/home/hp/Google_Paper/results/cifar100/{_SIZE_OF_DATASET_A}/ResNet110")

# Pretrain/Load the Student Model

In [21]:
if _SOFT_LABEL_DATASET_A:
  # If true then extend dataset_a with its examples soft-labeled by teacher.
  teacher_predictions = teacher_model.predict(data_split.dataset_a.examples)
  data_split.dataset_a.examples = tf.concat(
      (data_split.dataset_a.examples, data_split.dataset_a.examples), axis=0)
  data_split.dataset_a.labels = tf.concat(
      (data_split.dataset_a.labels, teacher_predictions), axis=0)

student_params = {
    'dataset': _DATASET,
    'epochs': _PRETRAINING_EPOCHS,
    'optimizer': _STUDENT_OPTIMIZER,
    'labeled_percentage': _SIZE_OF_DATASET_A,
    'architecture': _STUDENT_MODEL,
    'data_augmentation': _STUDENT_PRETRAINING_DATA_AUGMENTATION
    }

if _STUDENT_MODEL == 'mobilenet':
  student_params['depth'] = _STUDENT_MOBILENET_DEPTH_MULTIPLIER
elif _STUDENT_MODEL == 'resnet':
  student_params['depth'] = _STUDENT_RESNET_DEPTH

student_filename = 'STUDENT' + str(student_params)

student_filepath = os.path.join(_CNS_PATH, student_filename)

student_load_temp_filename = 'pretrained_weights.h5'
student_save_temp_filename = 'pretrained_weights.h5'

with strategy.scope():
  model_pretrained = load_model(
      _STUDENT_MODEL,
      num_classes,
      _STUDENT_OPTIMIZER,
      width_multiplier=1.0,
      depth_multiplier=_STUDENT_MOBILENET_DEPTH_MULTIPLIER,
      resnet_depth=_STUDENT_RESNET_DEPTH)

# if gfile.Exists(student_filepath):
#   gfile.Copy(student_filepath, student_load_temp_filename, overwrite=True)
#   with strategy.scope():
#     model_pretrained.load_weights(student_load_temp_filename)
# else:
train_model(
    model_pretrained,
    data_split.dataset_a.examples,
    data_split.dataset_a.labels,
    data_split.test.examples,
    data_split.test.labels,
    data_augmentation=_STUDENT_PRETRAINING_DATA_AUGMENTATION,
    epochs=_PRETRAINING_EPOCHS)

with strategy.scope():
  model_pretrained.save_weights(student_save_temp_filename)

  # If path is not empty then save pretrained student model.
  # if _CNS_PATH:
  #   gfile.Copy(student_save_temp_filename, student_filepath, overwrite=True)

scores = model_pretrained.evaluate(
    data_split.test.examples, data_split.test.labels, verbose=1)
print('pretrained model test-accuracy:', scores[1] * 100.0)

Epoch 1/200


2023-09-04 09:33:38.859641: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:9392"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

17/97 [====>.........................] - ETA: 6s - loss: 5.6215 - categorical_accuracy: 0.0299

97/97 [==============================] - 51s 144ms/step - loss: 5.2106 - categorical_accuracy: 0.0731 - val_loss: 5.2490 - val_categorical_accuracy: 0.0531 - lr: 0.0010
Epoch 2/200


2023-09-04 09:34:30.896317: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:9430"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
e

...

97/97 [==============================] - 13s 133ms/step - loss: 0.4733 - categorical_accuracy: 0.9998 - val_loss: 3.3492 - val_categorical_accuracy: 0.5200 - lr: 5.0000e-07
Epoch 199/200


2023-09-04 10:19:25.331291: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:16916"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


97/97 [==============================] - 13s 131ms/step - loss: 0.4732 - categorical_accuracy: 0.9998 - val_loss: 3.3488 - val_categorical_accuracy: 0.5193 - lr: 5.0000e-07
Epoch 200/200


2023-09-04 10:19:38.844848: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 12500
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:16954"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


313/313 [==============================] - 6s 17ms/step - loss: 3.3389 - categorical_accuracy: 0.5200
pretrained model test-accuracy: 51.99999809265137


# Estimate Teacher's Accuracy (Isotonic Regression)

Use either (normalized) entropy or (normalized) margin to estimate teacher's model accuracy.  We assume that the probability that a label from the teacher
is correct is monotone with respect to the margin and enforce this monotonicity
in our regression problem via Isotonic Regression.

In [22]:
import dataclasses
from typing import Callable, Optional

from sklearn.isotonic import IsotonicRegression
from sklearn.neighbors import KNeighborsRegressor


@dataclasses.dataclass
class Advice:
  """Class for advice used in distillation.

  Attributes
    ----------
    train: tf.Tensor
        A tensor containing the advice for the training data.
    validation: tf.Tensor
        A tensor containing the advice for the validation data.
    pretraining: tf.Tensor
        A tensor containing the advice for the pretraining data.
    test: tf.Tensor
        A tensor containing the advice for the test data.
  """
  train: Optional[tf.Tensor] = None
  validation: Optional[tf.Tensor] = None
  pretraining: Optional[tf.Tensor] = None
  test: Optional[tf.Tensor] = None


def margin(x: tf.Tensor,
           with_logits: Optional[bool] = False,
           normalize: Optional[bool] = False) -> tf.Tensor:
  """Computes the margin of a probability/logit tensor."""

  if with_logits:
    class_probabilities = tf.nn.softmax(x, axis=None, name=None)
  else:
    class_probabilities = x
  a, _ = tf.math.top_k(class_probabilities, k=2)
  marg = (tf.gather(a, [0], axis=1) - tf.gather(a, [1], axis=1))

  if normalize:
    marg = marg/tf.math.reduce_mean(marg)

  return marg


def entropy(x: tf.Tensor,
            with_logits: Optional[bool] = False,
            normalize: Optional[bool] = False) -> tf.Tensor:
  """Computes the entropy of a probability/logit tensor."""

  if with_logits:
    class_probabilities = tf.nn.softmax(x, axis=None, name=None)
  else:
    class_probabilities = x

  ent = tf.keras.losses.categorical_crossentropy(class_probabilities,
                                                 class_probabilities)
  if normalize:
    ent = ent/tf.math.reduce_mean(ent)

  return ent


def disagreement(q: tf.Tensor, p: tf.Tensor) -> tf.Tensor:
  """Returns a tensor with ones at the positions where p and q are different."""

  top_1_disagreement = tf.math.equal(tf.argmax(q, 1), tf.argmax(p, 1))
  return tf.cast(top_1_disagreement, tf.float32)


def _add_default_advice(data_split: DataSplit,
                        train_advice: tf.Tensor) -> Advice:
  """Adds the default advice for pretraining, validation, and test data."""

  pretraining = data_split.dataset_a
  validation = data_split.validation
  test = data_split.test

  validation_advice = None
  pretraining_advice = None

  if validation is not None:
    validation_advice = tf.ones(validation.size),
    validation_advice = tf.reshape(validation_advice, [-1])
  if pretraining is not None:
    pretraining_advice = tf.ones(pretraining.size),
    pretraining_advice = tf.reshape(pretraining_advice, [-1])

  test_advice = tf.zeros(test.size)
  test_advice = tf.reshape(test_advice, [-1])

  advice_data = Advice(
      train=train_advice,
      validation=validation_advice,
      pretraining=pretraining_advice,
      test=test_advice)

  return advice_data


def vanilla_advice(data_split: DataSplit) -> Advice:
  """A function that creates advice for vanilla distillation (used for testing).

  Args:
    data_split: Instance of datasets.UnlabeledDistillationData containing the
      training, validation, and test data.

  Returns:
    Instance of Advice containing advice vectors.
  """

  train = data_split.dataset_b
  main_advice = tf.ones(train.size)

  return _add_default_advice(data_split, main_advice)


def teacher_accuracy_advice_isotonic(
    teacher: tf.keras.Model,
    confidence: Callable[[tf.Tensor], tf.Tensor],
    data_split: DataSplit) -> Advice:
  """Creates teacher-accuracy advice for the training dataset (dataset_b).

  It uses a confidence measure (e.g., margin, entropy) for the teacher
  prediction over a validation dataset and isotonic regression to learn
  the mapping from confidence to accuracy.

  Args:
    teacher: Instance of tf.keras.Model: the teacher model.
    confidence: A function that maps soft labels to confidence.
    data_split: Instance of DataSplit containing the
      training, validation, and test data.

  Returns:
    Instance of Advice containing advice vectors.
  """

  train = data_split.dataset_b
  validation = data_split.validation

  # These are the lower bound/upper bounds used in isotonic regression.
  # All values predicted should be in the range [min_threshold, max_threshold].
  # Since the outputs correspond to the probability that the teacher model is
  # correct the range [0.5, 1.] is reasonable.
  min_threshold = .5
  max_threshold = 1.

  # Compute teacher predictions on the validation examples.
  teacher_predictions_validation = teacher.predict(validation.examples)

  # Compute the teacher margins on the validation examples.
  covariate = tf.reshape(confidence(teacher_predictions_validation), (-1, 1))

  # Compute the (pointwise) accuracy of the teacher on the validation data.
  response = disagreement(teacher_predictions_validation, validation.labels)

  # Create a isotonic regressor that maps teacher_margins to teacher_accuracies.
  covariate = tf.reshape(covariate, -1)
  response = tf.reshape(response, -1)
  iso = IsotonicRegression(
      y_min=min_threshold, y_max=max_threshold, out_of_bounds='clip')
  iso.fit(covariate, response)

  # Compute the teacher soft labels on the training dataset.
  teacher_predictions_training = teacher.predict(train.examples)

  # Compute the teacher confidence on the training dataset.
  teacher_confidence = tf.reshape(confidence(teacher_predictions_training), -1)

  # Use the knn to predict the accuracy of the teacher on the training examples.
  main_advice = iso.predict(teacher_confidence)

  return _add_default_advice(data_split, main_advice)


def teacher_accuracy_advice_from_entropy(
    teacher: tf.keras.Model,
    data_split: DataSplit,
    normalize: Optional[bool] = False) -> Advice:
  """Creates teacher-accuracy advice for the training dataset (dataset_b).

  Uses entropy as an uncertainty measure for the teacher and isotonic regression
  on the validation dataset.

  Args:
    teacher: Instance of tf.keras.Model: the teacher model.
    data_split: Instance of DataSplit containing the
      training, validation, and test data.
    normalize: True if the entropy should be normalized by the average entropy
    over the dataset.

  Returns:
    Instance of Advice containing advice vectors.
  """

  conf = lambda x: entropy(x, with_logits=False, normalize=normalize)

  return teacher_accuracy_advice_isotonic(teacher, conf, data_split)


def teacher_accuracy_advice_from_margin(
    teacher: tf.keras.Model,
    data_split: DataSplit,
    normalize: Optional[bool] = False) -> Advice:
  """Creates teacher-accuracy advice for the training dataset (dataset_b).

  Uses margin as an uncertainty measure for the teacher and isotonic regression
  on the validation dataset.

  Args:
    teacher: Instance of tf.keras.Model: the teacher model.
    data_split: Instance of DataSplit containing the
      training, validation, and test data.
    normalize: True if margin should be normalized by the average margin over
    the dataset.

  Returns:
    Instance of Advice containing advice vectors.
  """

  conf = lambda x: margin(x, with_logits=False, normalize=normalize)

  return teacher_accuracy_advice_isotonic(teacher, conf, data_split)


# Distillation Function

This is a simple wrapper function that takes the student model and the predictions of the teacher and trains the student.  It can take both advice
and weights to be used in distillation.

In [23]:
# from typing import TypedDict
from typing_extensions import TypedDict


# A simple dict for the various training parameters of the distill() function,
class TrainingParameters(TypedDict):
  with_soft_labels: Optional[bool]
  epochs: Optional[int]
  batch_size: Optional[int]
  data_augmentation: Optional[str]

def distill(model: tf.keras.Model,
             teacher_predictions: tf.Tensor,
             data_split: DataSplit,
             params: TrainingParameters,
             advice_data: Optional[Advice] = None,
             weight_data: Optional[Advice] = None) ->...:
  """A function that implements Distillation.

  Args:
    model: Instance of tf.keras.Model.
    teacher_predictions: Instance of tf.Tensor containing teacher's predictions.
    data_split: Instance of DataSplit.
    params: Instance of TrainingParameters typed dict that contains the
      following parameters:
      epochs - The number of training epochs.
      batch_size - The batch size used for training.
      with_soft_labels - If true use the soft-labels provided by the teacher.
      data_augmentation - One of None, 'offline', 'online'.
    advice_data: Instance of Advice to be used to as advice feature.
    weight_data: Instance of Advice to be used as weights for the training data.

  Returns:
    The achieved accuracy, the test-accuracy trajectory.
  """

  pretraining = data_split.dataset_a
  train = data_split.dataset_b
  validation = data_split.validation
  test = data_split.test

  # Default values for the parameters of the params dict.
  default_batch_size = 128
  default_epochs = 90
  default_with_soft_labels = True
  default_data_augmentation = None

  if params is not None:
    batch_size = params.get('batch_size', default_batch_size)
    epochs = params.get('epochs', default_epochs)
    with_soft_labels = params.get('with_soft_labels', default_with_soft_labels)
    data_augmentation = params.get('data_augmentation',
                                   default_data_augmentation)

  # We are doing soft-distillation by default.
  if with_soft_labels:
    targets = teacher_predictions
  else:
    arg_max_indices = tf.argmax(teacher_predictions, -1)
    targets = tf.keras.utils.to_categorical(arg_max_indices,
                                            len(teacher_predictions[0]))

  train_data = train.examples
  train_labels = tf.reshape(targets, tf.shape(train.labels))

  test_data = test.examples
  test_labels = test.labels

  # Include pretraining data in the training dataset
  if pretraining is not None and pretraining.trainable:
    train_data = tf.concat((pretraining.examples, train_data), axis=0)
    train_labels = tf.concat((pretraining.labels, train_labels), axis=0)

  # Include validation data in the training dataset
  if validation is not None and validation.trainable:
    train_data = tf.concat((train_data, validation.examples), axis=0)
    train_labels = tf.concat((train_labels, validation.labels), axis=0)

  # Fill advice for validation and pretraining with default values.
  if advice_data is not None:

    full_advice = advice_data.train

    if pretraining is not None and pretraining.trainable:
      full_advice = tf.concat((advice_data.pretraining, full_advice), axis=0)

    if validation is not None and validation.trainable:
      full_advice = tf.concat((full_advice, advice_data.validation), axis=0)

    if len(tf.shape(full_advice)) == 1:
      print("reshape")
      train_advice = tf.reshape(full_advice, shape=(-1, 1))
    else:
      train_advice = full_advice

    train_labels = tf.concat((train_labels, train_advice), axis=1)

    # We need dummy test advice in order for test data to be consistent
    # with the training data (we should always use zero for this).

    if len(tf.shape(advice_data.test)) == 1:
      test_advice = tf.reshape(advice_data.test, shape=(-1, 1))
    else:
      test_advice = advice_data.test
    test_labels = tf.concat((test.labels, test_advice), axis=1)

  # Use no weights by default.
  weights = None
  if weight_data is not None:

    full_weights = weight_data.train

    if pretraining is not None and pretraining.trainable:
      full_weights = tf.concat((weight_data.pretraining, full_weights), axis=0)

    if validation is not None and validation.trainable:
      full_weights = tf.concat((full_weights, weight_data.validation), axis=0)

    weights = tf.reshape(full_weights, [-1])

  score = model.evaluate(test.examples, test_labels)
  if isinstance(score, (list, tuple, np.ndarray)):
    score = score[1]

  history = train_model(
      model,
      train_data,
      train_labels,
      test_data,
      test_labels,
      data_augmentation=data_augmentation,
      weights=weights,
      epochs=epochs,
      batch_size=batch_size)

  trajectory = np.array(history.history['val_categorical_accuracy'])
  trajectory = np.insert(trajectory, 0, score)
  trajectory *= 100.0
  result = np.max(trajectory)

  return (result, trajectory)


# Randomize dataset B and create validation data.

In [24]:
# Create a small validation dataset.
if _RANDOMIZE_DATASET:
  data_split.dataset_b.shuffle(_SEED)

# Round the size of validation to be multiple of batch_size
rounded_validation_dataset_size = _BATCH_SIZE * round(
    _VALIDATION_DATASET_SIZE / _BATCH_SIZE)

train, validation = split(data_split.dataset_b, rounded_validation_dataset_size)

train.trainable = True

# We will train the student model on the validation dataset.
validation.trainable = _TRAIN_ON_VALIDATION

# Update the data_split datasets with the new training and validation data.
data_split.dataset_b = train
data_split.validation = validation

# Predict the Labels on the unlabeled dataset using the teacher model
teacher_predictions = teacher_model.predict(data_split.dataset_b.examples)

 636/1156 [===============>..............] - ETA: 11s

1156/1156 [==============================] - 29s 22ms/step


# Vanilla Student Mixing Loss

The student mixing decorator gets a loss function and returns the student-label-mixing loss.  We use it primarily with the categorical cross-entropy loss function.

In [25]:
def advice_mix(loss: Callable[[tf.Tensor, tf.Tensor], tf.Tensor]
               ) -> Callable[[tf.Tensor, tf.Tensor], tf.Tensor]:
  """A decorator that returns a student label mixing loss function.

  Args:
      loss: A loss function that takes two tensors as arguments, e.g., the
        tf.keras.losses.CategoricalCrossentropy().

  Returns:
    A loss that takes tensors with advice and computes the student mixing loss.
  """

  def _loss(y_true, y_pred):

    # Split y_true into the advice (last column of y_true) and labels.
    advice = tf.stop_gradient(y_true[:, -1:])
    label = tf.stop_gradient(y_true[:, :-1])

    # Mix the student label (y_pred) using the advice.
    y_mixed = y_pred * advice + (1. - y_pred) * (1 - advice)

    # Return the loss of the teacher label and the mixed student label.
    return loss(label, y_mixed)

  return _loss


def mixed_cce(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
  """Computes the cross entropy loss using the advice to perform student mixing.

  Args:
    y_true: tf.Tensor with num_classes + 1 columns where the first num_classes
      columns contain the true label and the last column corresponds to the
      advice.
    y_pred: tf.Tensor with num_classes columns that contains the predicted
      probabilities for each class.

  Returns:
    The student mixed cross entropy between y_true[:,:-1] and y_pred defined as:
    ce(teacher_pred, advice student_pred + (1-advice) (1-student_pred))
  """

  # We have to use tf.keras.losses.Reduction.NONE when we are using
  # tf.distribute.Strategy.
  # (see https://www.tensorflow.org/tutorials/distribute/custom_training).
  loss = advice_mix(
      tf.keras.losses.CategoricalCrossentropy(
          reduction=tf.keras.losses.Reduction.NONE))

  return tf.math.reduce_mean(loss(y_true, y_pred))


# Top-k Student Label Mixing loss

In [26]:
import tensorflow_probability as tfp
def soft_top_k(x, k, temp):
  perm = tfp.math.soft_sorting_matrix(x, temp)[:,:k]
  return tf.reduce_sum(perm, axis=1)


def advice_mix_soft_top_k(k, temperature):
  """A decorator that returns a top_k student label mixing loss decorator.

  Args:
      k: the top_k elements of the student to be used.
      temperature: the smoothing temperature of the top_k operator.

  Returns:
    A top_k student label mixing loss decorator.
    loss.
  """

  def _advice_mix_soft_top_k(loss):

    def _loss(y_true, y_pred):

      # Split y_true into the advice (last column of y_true) and labels.
      advice = tf.stop_gradient(y_true[:, -1:])
      label = tf.stop_gradient(y_true[:, :-1])

      # boost = tf.stop_gradient(soft_top_k(y_pred, k, temperature))
      boost = soft_top_k(y_pred, k, temperature)

      missing = tf.stop_gradient((1. - boost) * (1. - y_pred))
      y_mixed = y_pred * advice + (1.-advice) * (boost * (1.-y_pred) + missing)

      # Return the loss of the teacher label and the mixed student label.
      return loss(label, y_mixed)

    return _loss

  return _advice_mix_soft_top_k



def mixed_cce_top_k(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
  """Computes the cross entropy loss using the advice to perform student mixing.

  Args:
    y_true: tf.Tensor with num_classes + 1 columns where the first num_classes
      columns contain the true label and the last column corresponds to the
      advice.
    y_pred: tf.Tensor with num_classes columns that contains the predicted
      probabilities for each class.

  Returns:
    The student mixed cross entropy between y_true[:,:-1] and y_pred defined as:
    ce(teacher_pred, advice student_pred + (1-advice) (1-student_pred))
  """

  # We have to use tf.keras.losses.Reduction.NONE when we are using
  # tf.distribute.Strategy.
  # (see https://www.tensorflow.org/tutorials/distribute/custom_training).
  mixing_operator = advice_mix_soft_top_k(20, 100.0)
  loss = mixing_operator(tf.keras.losses.CategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE))

  return tf.math.reduce_mean(loss(y_true, y_pred))

In [27]:
# Compute the advice data using margin.
advice_data = teacher_accuracy_advice_from_margin(teacher_model, data_split)
advice_data.train

1156/1156 [==============================] - 26s 22ms/step


array([0.5       , 0.69230769, 0.69230769, ..., 0.6       , 0.5       ,
       0.5       ])

In [28]:
# Define the training parameters for the distill() funcion.
training_params = {
    'epochs': _STUDENT_EPOCHS,
    'with_soft_labels': _WITH_SOFT_LABELS,
    'batch_size': _BATCH_SIZE,
    'data_augmentation': _STUDENT_DISTILLATION_DATA_AUGMENTATION
}

In [29]:
# Load a fresh student model with the studentl-label-mixing cross-entropy loss.
with strategy.scope():
  student = load_model(
      _STUDENT_MODEL,
      num_classes=num_classes,
      optimizer_name=_STUDENT_OPTIMIZER,
      loss_function=mixed_cce_top_k,
      width_multiplier=1,
      depth_multiplier=_STUDENT_MOBILENET_DEPTH_MULTIPLIER,
      resnet_depth=_STUDENT_RESNET_DEPTH)

  student.load_weights('pretrained_weights.h5')

In [30]:
# Train the student model using distillation.
distill(model=student,
        teacher_predictions=teacher_predictions,
        data_split=data_split,
        advice_data=advice_data,
        params=training_params)


reshape
313/313 [==============================] - 8s 15ms/step - loss: 7.8124 - categorical_accuracy: 0.5200
Epoch 1/100


2023-09-04 10:21:09.651508: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:17095"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 101
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 73s 94ms/step - loss: 3.4881 - categorical_accuracy: 0.4748 - val_loss: 6.9210 - val_categorical_accuracy: 0.4124 - lr: 0.0010
Epoch 2/100


2023-09-04 10:22:24.248556: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:17133"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 101
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 35s 90ms/step - loss: 3.4286 - categorical_accuracy: 0.5071 - val_loss: 6.7149 - val_categorical_accuracy: 0.3710 - lr: 0.0010
Epoch 3/100


...

2023-09-04 12:42:10.515149: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:20819"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 101
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 36s 91ms/step - loss: 2.9019 - categorical_accuracy: 0.7333 - val_loss: 8.8094 - val_categorical_accuracy: 0.5770 - lr: 1.0000e-04
Epoch 100/100


2023-09-04 12:42:47.794760: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:20857"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 101
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 37s 93ms/step - loss: 2.9029 - categorical_accuracy: 0.7253 - val_loss: 8.8216 - val_categorical_accuracy: 0.5738 - lr: 1.0000e-04


(58.85000228881836,
 array([51.99999809, 41.24000072, 37.09999919, 41.10000134, 41.02999866,
        43.07000041, 42.05000103, 44.08000112, 46.23999894, 45.62999904,
        46.2199986 , 46.30999863, 43.88999939, 40.84999859, 44.08999979,
        41.87000096, 46.09000087, 45.44999897, 43.05000007, 45.15999854,
        45.10999918, 49.68000054, 45.60000002, 47.13000059, 48.03000093,
        46.68000042, 49.14999902, 43.84999871, 47.15999961, 45.19000053,
        47.88999856, 45.62000036, 49.16000068, 46.63000107, 50.11000037,
        50.19000173, 47.81999886, 47.69000113, 48.46000075, 47.02000022,
        47.0600009 , 47.78000116, 46.00999951, 46.6899991 , 48.33000004,
        48.39000106, 45.51999867, 52.02999711, 49.32999909, 46.00999951,
        48.26999903, 49.57000017, 47.38000035, 48.05999994, 48.82999957,
        48.69999886, 47.67999947, 50.42999983, 48.66999984, 46.63000107,
        49.39999878, 47.87999988, 48.6499995 , 47.47999907, 49.36999977,
        48.60000014, 50.7700026

# Vanilla Distillation

In [31]:
# Load a fresh student model with the standard cross-entropy loss.
with strategy.scope():
  vanilla_student = load_model(
      _STUDENT_MODEL,
      num_classes=num_classes,
      optimizer_name=_STUDENT_OPTIMIZER,
      width_multiplier=1,
      depth_multiplier=_STUDENT_MOBILENET_DEPTH_MULTIPLIER,
      resnet_depth=_STUDENT_RESNET_DEPTH)

  vanilla_student.load_weights('pretrained_weights.h5')

In [32]:
# Train the student model using distillation.
distill(model=vanilla_student,
        teacher_predictions=teacher_predictions,
        data_split=data_split,
        advice_data=None,
        params=training_params)

313/313 [==============================] - 8s 18ms/step - loss: 3.3389 - categorical_accuracy: 0.5200
Epoch 1/100


2023-09-04 12:43:36.768699: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:20916"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


 31/390 [=>............................] - ETA: 28s - loss: 2.6725 - categorical_accuracy: 0.5736

390/390 [==============================] - 73s 90ms/step - loss: 2.0769 - categorical_accuracy: 0.6028 - val_loss: 2.9804 - val_categorical_accuracy: 0.4463 - lr: 0.0010
Epoch 2/100


2023-09-04 12:44:51.284775: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:20954"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


...

2023-09-04 13:45:11.340733: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:24640"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 35s 89ms/step - loss: 0.8291 - categorical_accuracy: 0.9269 - val_loss: 2.2668 - val_categorical_accuracy: 0.5654 - lr: 1.0000e-04
Epoch 100/100


2023-09-04 13:45:47.760044: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:24678"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 36s 92ms/step - loss: 0.8143 - categorical_accuracy: 0.9296 - val_loss: 2.2536 - val_categorical_accuracy: 0.5639 - lr: 1.0000e-04


(56.99999928474426,
 array([51.99999809, 44.63      , 44.78000104, 44.49000061, 47.45999873,
        44.15999949, 50.48000216, 47.06999958, 48.28999937, 48.39000106,
        50.80000162, 45.60000002, 49.86999929, 49.27999973, 46.9900012 ,
        46.61000073, 45.1700002 , 48.46999943, 47.38999903, 48.42000008,
        48.12000096, 49.88999963, 49.61999953, 48.59000146, 50.06999969,
        50.2399981 , 48.66999984, 49.16999936, 49.09999967, 50.67999959,
        50.48000216, 47.20999897, 52.56999731, 50.80000162, 47.47000039,
        51.24999881, 50.90000033, 51.81999803, 50.04000068, 50.5400002 ,
        51.27999783, 50.62000155, 52.10999846, 51.52999759, 51.49999857,
        51.27000213, 49.57000017, 50.72000027, 47.2600013 , 48.80000055,
        50.63999891, 49.1899997 , 50.3000021 , 48.46999943, 51.06999874,
        50.91999769, 49.84000027, 51.26000047, 49.02000129, 49.55999851,
        50.66000223, 51.16000175, 51.3800025 , 51.21999979, 47.20000029,
        49.39999878, 52.0500004

# Weighted Distillation

In [33]:
def accuracy_distortion_weights(teacher: tf.keras.Model,
                                student: tf.keras.Model,
                                uncertainty: Callable[[tf.Tensor], tf.Tensor],
                                n_neighbors: int,
                                data_split: DataSplit) -> Advice:
  """Args: Maps (teacher, model)-confidence to (accuracy, distortion) pairs.

  Args:
    teacher: Instance of tf.keras.Model.
    student: Instance of tf.keras.Model.
    uncertainty: A function that maps probability distributions to uncertainty.
    n_neighbors: The number of neighbors to average over when using 'Nearest
      Neighbors' to learn the weights.
    data_split: Instance of DataSplit containing the data.

  Returns:
    An instance of Advice containing advice for the examples.
  """
  clip_max = 100000
  clip_min = .00001

  train = data_split.dataset_b
  validation = data_split.validation

  student_predictions_validation = student.predict(validation.examples)
  teacher_predictions_validation = teacher.predict(validation.examples)

  teacher_uncertainty_validation = tf.reshape(
      uncertainty(teacher_predictions_validation), (-1, 1))

  student_uncertainty_validation = tf.reshape(
      uncertainty(student_predictions_validation), (-1, 1))

  cce = tf.keras.losses.CategoricalCrossentropy(
      reduction=tf.keras.losses.Reduction.NONE)

  noisy_loss = cce(teacher_predictions_validation,
                   student_predictions_validation)

  clean_loss = cce(validation.labels, student_predictions_validation)

  distortion_validation = noisy_loss / (clean_loss + clip_min)

  accuracy_validation = tf.cast(
      tf.math.equal(
          tf.argmax(teacher_predictions_validation, 1),
          tf.argmax(validation.labels, 1)), tf.float32)

  accuracy_validation = tf.reshape(accuracy_validation, shape=(-1, 1))
  distortion_validation = tf.reshape(distortion_validation, shape=(-1, 1))

  # If accuracy is 1 we want to set distortion to be 1 too.
  # To do this we first multiply distortion by (1.- accuracy_validation)
  # in order to zero-out all elements whose corresponding teacher predictions
  # are correct.
  distortion_validation_masked = distortion_validation * (
      1. - accuracy_validation)

  # Replace the distortion of the masked elements with their accuracy
  # (which is equal to 1).
  distortion_validation = distortion_validation_masked + accuracy_validation

  # Clip distortion values for numerical stability.
  distortion_validation_clipped = tf.clip_by_value(
      distortion_validation, clip_value_min=0., clip_value_max=clip_max)

  # We create the features and responses of the 2-dimension regression problem.
  # The features are pairs (teacher_uncertainty, student_uncertainty) and the
  # responses are pairs (accuracy, distortion).
  space = tf.concat(
      (teacher_uncertainty_validation, student_uncertainty_validation), axis=1)
  y = tf.concat((accuracy_validation, distortion_validation_clipped), axis=1)

  # Use knn to solve the regression problem.
  neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
  neigh.fit(space, y)

  # Compute the predictions of the teacher model on the training examples.
  teacher_predictions_train = teacher.predict(train.examples)
  # Compute the uncertainty of teacher's predictions.
  teacher_uncertainty_train = tf.reshape(
      uncertainty(teacher_predictions_train), shape=(-1, 1))

  # Compute the predictions of the student model on the training examples.
  student_train_predictions = student.predict(train.examples)
  # Compute the uncertainty of student's predictions.
  student_uncertainty_train = tf.reshape(
      uncertainty(student_train_predictions), shape=(-1, 1))

  # Create the uncertainty features over the training data.
  uncertainty_train = tf.concat(
      (teacher_uncertainty_train, student_uncertainty_train), axis=1)

  # Use the knn to obtain (accuracy, distortion) predictions over the training
  # examples.
  accuracy_distortion = neigh.predict(uncertainty_train)

  accuracy_predicted = accuracy_distortion[:, :1]
  distortion_predicted = accuracy_distortion[:, 1:]

  # Compute the debiasing weights as described in the paper
  # https://drive.google.com/file/d/1ye5PbhDHbu4Pr8jFskGVt_XzyPB9DYez/view?resourcekey=0-TPAMM5WVZ2IdrmyIBitctw.
  weight = 1. / (1. + (1. - accuracy_predicted) * (distortion_predicted - 1))
  weight = tf.clip_by_value(weight, clip_value_min=0., clip_value_max=1.)
  weight = tf.reshape(weight, shape=[-1])

  return _add_default_advice(data_split, weight)


In [34]:
# Load a fresh student model with the standard cross-entropy loss.
with strategy.scope():
  acc_dist_student = load_model(
      _STUDENT_MODEL,
      num_classes=num_classes,
      optimizer_name=_STUDENT_OPTIMIZER,
      width_multiplier=1,
      depth_multiplier=_STUDENT_MOBILENET_DEPTH_MULTIPLIER,
      resnet_depth=_STUDENT_RESNET_DEPTH)

  acc_dist_student.load_weights('pretrained_weights.h5')

validation_size = data_split.validation.size
# The dimension of the covariate space of the KNN is d = 2.
d = 2
# We use the asymptotic formula to compute the number of neighbors.
n_neigh = round(0.5 * (validation_size)**(2 / (2 + d)))

def _margin(x):
  return margin(x, with_logits=False, normalize=False)

weight_data = accuracy_distortion_weights(
    teacher=teacher_model,
    student=acc_dist_student,
    uncertainty=_margin,
    n_neighbors=n_neigh,
    data_split=data_split)

# Perform weighted distillation.
distill(
    model=acc_dist_student,
    teacher_predictions=teacher_predictions,
    data_split=data_split,
    weight_data=weight_data,
    params=training_params)

 63/313 [=====>........................] - ETA: 4s - loss: 3.3301 - categorical_accuracy: 0.5154

313/313 [==============================] - 8s 18ms/step - loss: 3.3389 - categorical_accuracy: 0.5200
Epoch 1/100


2023-09-04 13:47:23.636720: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:24817"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
      shape {
      }
    }
  }

390/390 [==============================] - 73s 96ms/step - loss: 1.5520 - categorical_accuracy: 0.5893 - val_loss: 3.0628 - val_categorical_accuracy: 0.4404 - lr: 0.0010
Epoch 2/100


2023-09-04 13:48:37.641173: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:24855"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
      shape {
      }
    }
  }

...

390/390 [==============================] - 37s 95ms/step - loss: 0.9097 - categorical_accuracy: 0.7559 - val_loss: 2.8738 - val_categorical_accuracy: 0.4918 - lr: 0.0010
Epoch 52/100


2023-09-04 14:20:30.246997: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:26755"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
      shape {
      }
    }
  }

390/390 [==============================] - 37s 94ms/step - loss: 0.9272 - categorical_accuracy: 0.7468 - val_loss: 2.6371 - val_categorical_accuracy: 0.5123 - lr: 0.0010
Epoch 53/100


2023-09-04 14:21:08.478600: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:26793"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 100
        }
      }
      shape {
      }
    }
  }

390/390 [==============================] - ETA: 0s - loss: 0.9007 - categorical_accuracy: 0.7612

# SLaM (Data dependent top-k)

In [ ]:
def top_k_disagreement(q: tf.Tensor, p: tf.Tensor, k: int) -> tf.Tensor:
  num_classes = tf.shape(p)[-1]
  _, top_k_pos = tf.math.top_k(p, k=k)
  one_hot_top_k = tf.reduce_sum(tf.one_hot(top_k_pos,  num_classes), axis=1)
  true_top_1 = tf.one_hot(tf.argmax(q, 1), num_classes)

  return tf.reduce_sum(one_hot_top_k * true_top_1, axis=1)


def top_k_margin(x: tf.Tensor,
                k: int,
                with_logits: Optional[bool] = False,
                normalize: Optional[bool] = False) -> tf.Tensor:
  """Computes the margin of a probability/logit tensor."""

  if with_logits:
    class_probabilities = tf.nn.softmax(x, axis=None, name=None)
  else:
    class_probabilities = x
  a, _ = tf.math.top_k(class_probabilities, k=k+1, sorted=True)

  # Compute total probability of top-k elements
  top_k_prob = tf.reduce_sum(a[:, :k], axis=1)
  next_best_prob =  tf.reduce_sum(a[:, k:], axis=1)
  marg = top_k_prob - next_best_prob

  if normalize:
    marg = marg/tf.math.reduce_mean(marg)

  return marg


def teacher_top_k_accuracy_predictions(
    teacher_predictions_training: tf.keras.Model,
    teacher_predictions_validation: tf.keras.Model,
    k: int,
    confidence: Callable[[tf.Tensor], tf.Tensor],
    data_split: DataSplit) -> tf.Tensor:
  """Creates teacher-accuracy advice for the training dataset (dataset_b).

  It uses a confidence measure (e.g., margin, entropy) for the teacher
  prediction over a validation dataset and isotonic regression to learn
  the mapping from confidence to accuracy.

  Args:
    teacher: Instance of tf.keras.Model: the teacher model.
    confidence: A function that maps soft labels to confidence.
    k: Compute the top-k accuracy of teacher.
    data_split: Instance of DataSplit containing the
      training, validation, and test data.

  Returns:
    Instance of Advice containing advice vectors.
  """

  train = data_split.dataset_b
  validation = data_split.validation

  # These are the lower bound/upper bounds used in isotonic regression.
  # All values predicted should be in the range [min_threshold, max_threshold].
  # Since the outputs correspond to the probability that the teacher model is
  # correct the range [0.5, 1.] is reasonable.
  min_threshold = 0.5
  max_threshold = 1.

  # Compute the teacher margins on the validation examples.
  covariate = tf.reshape(confidence(teacher_predictions_validation), (-1, 1))

  # Compute the (pointwise) top-k accuracy of the teacher on the validation data.
  response = top_k_disagreement(validation.labels, teacher_predictions_validation, k)

  # Create a isotonic regressor that maps teacher_margins to teacher_accuracies.
  covariate = tf.reshape(covariate, -1)
  response = tf.reshape(response, -1)
  iso = IsotonicRegression(
      y_min=min_threshold, y_max=max_threshold, out_of_bounds='clip')
  iso.fit(covariate, response)


  # Compute the teacher confidence on the training dataset.
  teacher_confidence = tf.reshape(confidence(teacher_predictions_training), -1)

  # Use the knn to predict the accuracy of the teacher on the training examples.
  main_advice = iso.predict(teacher_confidence)

  return main_advice

def teacher_top_k_accuracy_advice(
    teacher: tf.keras.Model,
    num_classes,
    data_split: DataSplit) -> Advice:

  pretraining = data_split.dataset_a
  validation = data_split.validation
  test = data_split.test

  s = []

  teacher_predictions_training = teacher.predict(train.examples)
  teacher_predictions_validation = teacher.predict(validation.examples)

  for k in range(1, num_classes):

    def _top_k_confidence(x):
      return top_k_margin(x, k, with_logits=False, normalize=False)

    s.append(teacher_top_k_accuracy_predictions(
        teacher_predictions_training,
        teacher_predictions_validation,
        k, _top_k_confidence, data_split))

  s.append(tf.ones(data_split.dataset_b.size))

  train_advice = tf.stack(s, axis=1)

  validation_advice = None
  pretraining_advice = None

  if validation is not None:
    validation_advice = tf.stack([tf.one_hot(0, num_classes) for i in range(validation.size)], axis=0)

  if pretraining is not None:
    pretraining_advice =  tf.stack([tf.one_hot(0, num_classes) for i in range(pretraining.size)], axis=0)

  test_advice =  tf.stack([tf.zeros(num_classes) for i in range(test.size)], axis=0)

  advice_data = Advice(
      train=train_advice,
      validation=validation_advice,
      pretraining=pretraining_advice,
      test=test_advice)

  return advice_data


In [ ]:
full_top_k_advice = teacher_top_k_accuracy_advice(teacher_model, 100, data_split)

16/16 [==============================] - 0s 16ms/step


In [ ]:
import tensorflow_probability as tfp

def soft_top_k_mask(x, k_mask, temp):
  perm_matrices =  tfp.math.soft_sorting_matrix(x, temp)
  return tf.einsum('ijk, ij -> ik', perm_matrices, k_mask)


def advice_mix_soft_top_full(num_classes, temperature, threshold):
  """A decorator that returns a top_k student label mixing loss decorator.

  Args:
      k: the top_k elements of the student to be used.

  Returns:
    A top_k student label mixing loss decorator.
    loss.
  """

  def _advice_mix_soft_top_k(loss):

    def _loss(y_true, y_pred):

      # Split y_true into the advice (last column of y_true) and labels.
      advice = tf.stop_gradient(y_true[:, -num_classes:])
      label = tf.stop_gradient(y_true[:, :-num_classes])
      k_mask = tf.cast(advice<threshold, tf.float32)
      boost = soft_top_k_mask(y_pred, k_mask, temperature) * (1-y_pred)

      top_1_acc = advice[:, 0]
      top_1_err = 1.-top_1_acc

      y_1 = tf.einsum('ij, i -> ij', y_pred, top_1_acc)
      y_2  = tf.einsum('ij, i -> ij', boost, top_1_err)
      y_mixed = y_1 + y_2

      # Return the loss of the teacher label and the mixed student label.
      return loss(label, y_mixed)

    return _loss

  return _advice_mix_soft_top_k


def mixed_cce_soft_top_full(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
  """Computes the cross entropy loss using the advice to perform student mixing.

  Args:
    y_true: tf.Tensor with num_classes + 1 columns where the first num_classes
      columns contain the true label and the last column corresponds to the
      advice.
    y_pred: tf.Tensor with num_classes columns that contains the predicted
      probabilities for each class.

  Returns:
    The student mixed cross entropy between y_true[:,:-1] and y_pred defined as:
    ce(teacher_pred, advice student_pred + (1-advice) (1-student_pred))
  """

  # We have to use tf.keras.losses.Reduction.NONE when we are using
  # tf.distribute.Strategy.
  # (see https://www.tensorflow.org/tutorials/distribute/custom_training).
  mixing_operator = advice_mix_soft_top_full(100, 100.0, 0.9)
  loss = mixing_operator(tf.keras.losses.CategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE))

  return tf.math.reduce_mean(loss(y_true, y_pred))


In [ ]:
# Load a fresh student model with the studentl-label-mixing cross-entropy loss.
with strategy.scope():
  student_soft_full = load_model(
      _STUDENT_MODEL,
      num_classes=num_classes,
      optimizer_name=_STUDENT_OPTIMIZER,
      loss_function=mixed_cce_soft_top_full,
      width_multiplier=1,
      depth_multiplier=_STUDENT_MOBILENET_DEPTH_MULTIPLIER,
      resnet_depth=_STUDENT_RESNET_DEPTH)

  student_soft_full.load_weights('pretrained_weights.h5')

In [ ]:
# Train the student model using distillation.
distill(model=student_soft_full,
        teacher_predictions=teacher_predictions,
        data_split=data_split,
        advice_data=full_top_k_advice,
        params=training_params)


313/313 [==============================] - 6s 11ms/step - loss: 7.7357 - categorical_accuracy: 0.4761
Epoch 1/100


2023-09-03 12:49:00.470331: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:26938"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 200
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 64s 74ms/step - loss: 3.1287 - categorical_accuracy: 0.1615 - val_loss: 6.8793 - val_categorical_accuracy: 0.3732 - lr: 0.0010
Epoch 2/100


2023-09-03 12:50:05.845758: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:26976"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 200
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 28s 71ms/step - loss: 3.0694 - categorical_accuracy: 0.1564 - val_loss: 7.0704 - val_categorical_accuracy: 0.4004 - lr: 0.0010
Epoch 3/100


...

2023-09-03 13:38:07.541114: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:30662"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 200
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 28s 71ms/step - loss: 2.2438 - categorical_accuracy: 0.2132 - val_loss: 8.2659 - val_categorical_accuracy: 0.5247 - lr: 1.0000e-04
Epoch 100/100


2023-09-03 13:38:36.904868: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:30700"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 200
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}


390/390 [==============================] - 29s 73ms/step - loss: 2.2359 - categorical_accuracy: 0.2136 - val_loss: 8.2760 - val_categorical_accuracy: 0.5267 - lr: 1.0000e-04


(53.96000146865845,
 array([47.60999978, 37.31999993, 40.04000127, 38.82000148, 39.32999969,
        43.27000082, 40.56999981, 42.21000075, 43.61000061, 40.7099992 ,
        42.71000028, 40.25000036, 42.87      , 43.88999939, 44.33000088,
        43.1400001 , 42.23000109, 45.78000009, 45.7100004 , 43.23000014,
        44.76999938, 40.92000127, 42.95000136, 45.12000084, 46.09000087,
        47.60999978, 45.93000114, 47.69000113, 44.92999911, 45.50000131,
        44.98000145, 41.3500011 , 44.51000094, 40.63000083, 45.57000101,
        43.97999942, 46.68000042, 44.24999952, 44.83999908, 46.39999866,
        44.63      , 46.450001  , 46.7200011 , 47.36999869, 47.2299993 ,
        45.12000084, 47.08999991, 45.28000057, 47.88999856, 45.96000016,
        46.81000113, 44.94999945, 47.36000001, 47.54999876, 46.2500006 ,
        46.23999894, 45.26000023, 46.07000053, 48.0399996 , 46.05999887,
        43.59999895, 45.30000091, 43.52000058, 47.11999893, 48.48999977,
        45.39999962, 46.1800009